# Changes in QARTOD Gross Range and Climatology Lookup Tables

The lookup tables for gross range and climatology QARTOD tests on OOI CTDBP data were last updated on 12 Jan 2022, and the "suspect" ranges were calculated with all data before the cutoff date of 31 Dec 2021. However, when we recalculate these test ranges now, the values are different because recovered data that was recorded by the sensor before the cutoff date were recovered and ingested after the current lookup tables were published online. 

Deployment 14 of the Coastal Endurance array was the last deployment to be recovered before the cutoff date on 17 Sept 2021, so this is the last date that recovered instrument data was available for the original lookup tables. We can assume that deployment 14 data was included in the lookup table calculation since it was ingested on 18 Oct 2021. Since deployment 15 data was recovered 31 Mar 2022 and ingested xx xxx 2022, any gross range or climatology test ranges recalculated after 2023 with the same cutoff date will include additional recovered data from 17 Sept to 31 Dec 2021.

(Note: We could change the cutoff date to 17 Sept 2021 when calling on the script to recalculate the lookup tables, and see if that would result in the same lookup table values.)

In [1]:
# Load python modules used in this notebook into the workspace
import ooi_data_explorations.qartod.qc_processing as qc_process
import ooi_data_explorations.common as ooi_tools
from ooi_data_explorations.common import load_gc_thredds
from ooi_data_explorations.uncabled.process_ctdbp import ctdbp_datalogger, ctdbp_instrument

from ooinet import M2M

import numpy as np
import pandas as pd
import xarray as xr
import dask

In [2]:
# set parameters for a particular sensor
site = 'CE01ISSM'
node = 'SBD17'
sensor = '06-CTDBPC000'

refdes = '-'.join([site,node,sensor])

In [3]:
vocab = M2M.get_vocab(refdes)
vocab

,@class,vocabId,refdes,instrument,tocL1,tocL2,tocL3,manufacturer,model,mindepth,maxdepth
0,.VocabRecord,34,CE01ISSM-SBD17-06-CTDBPC000,CTD,Coastal Endurance,Oregon Inshore Surface Mooring,Surface Buoy,Sea-Bird,SBE 16plusV2,1.0,1.0


In [4]:
deployments = M2M.get_deployments(refdes)
deployments

,deploymentNumber,uid,assetId,latitude,longitude,depth,deployStart,deployEnd,deployCruise,recoverCruise
0,2,CGINS-CTDBPC-50011,1542,44.65833,-124.09583,1.0,2014-10-10 17:45:00,2015-04-12 00:30:00,OC1410A,None
1,4,CGINS-CTDBPC-50154,3419,44.6601,-124.09582,1.0,2015-10-08 19:03:00,2016-05-10 15:22:00,OC1510A,None
2,6,CGINS-CTDBPC-50188,3329,44.65975,-124.09504,1.0,2016-09-30 16:45:00,2017-04-17 19:45:00,AT37-03,None
3,7,CGINS-CTDBPC-50151,3416,44.65628,-124.09522,1.0,2017-04-19 04:18:00,2017-10-12 21:50:57,SKQ201704S,None
4,8,CGINS-CTDBPC-50011,1542,44.6597,-124.09486,1.0,2017-10-12 17:50:38,2018-04-01 10:22:00,SKQ201715S,None
5,9,CGINS-CTDBPC-07240,3603,44.65685,-124.09553,1.0,2018-04-03 04:22:00,2018-09-30 15:47:00,SKQ201808S,None
6,10,CGINS-CTDBPC-50187,3327,44.65975,-124.09504,1.0,2018-09-29 20:30:00,2019-04-27 16:04:00,SR1813,None
7,11,CGINS-CTDBPC-50057,1487,44.65683,-124.09533,1.0,2019-04-20 03:01:00,2019-10-22 23:57:00,SKQ201910S,None
8,12,CGINS-CTDBPC-50015,1550,44.65983,-124.09543,1.0,2019-10-20 23:02:00,2020-07-10 15:25:00,SKQ201921S,None
9,13,CGINS-CTDBPC-50007,1539,44.65678,-124.09707,1.0,2020-07-13 16:06:00,2021-03-28 17:00:00,TN380,None


In [ ]:
metadata = M2M.get_metadata(refdes)
metadata # all this tells me is that ingestion_timestamp exists as a variable for the data streams telemetered, recovered_host, and recovered_inst

In [ ]:
# download recovered_host data on last day of deployment 10
tag = '.*CTDBP.*\\.nc$'
method = 'recovered_host'
stream = 'ctdbp_cdef_dcl_instrument_recovered'
req_host = ooi_tools.m2m_request(site, node, sensor, method, stream, start='2021-09-17T00:00:00.000Z', stop='2021-09-17T14:00:00.000Z')
data_host = ooi_tools.m2m_collect(req_host, tag)
data_host
# added data ingestion date in notes on OOI Data Explorations

In [ ]:
# download recovered_inst data on last day of deployment 10
tag = '.*CTDBP.*\\.nc$'
method = 'recovered_inst'
stream = 'ctdbp_cdef_instrument_recovered'
req_inst = ooi_tools.m2m_request(site, node, sensor, method, stream, start='2021-09-17T00:00:00.000Z', stop='2021-09-17T14:00:00.000Z')
data_inst = ooi_tools.m2m_collect(req_inst, tag)
data_inst
# added data ingestion date in notes on OOI Data Explorations